In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,081 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,004 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,269 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,342 kB]
Hit:13 https://ppa.launchpadcontent.n

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [31]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

df.createOrReplaceTempView("home_sales")
df.show(10)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [33]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    YEAR(date)
ORDER BY
    year
"""

result = spark.sql(query)
result.show()



+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [34]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query and show the results
result = spark.sql(query)
result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [35]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query and display the results
result = spark.sql(query)
result.show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [19]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = """
SELECT
    `view` AS view_rating,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    price >= 350000
GROUP BY
    `view`
"""
# Execute the query and display the results
result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|         31| 399856.95|
|         85|1056336.74|
|         65| 736679.93|
|         53|  755214.8|
|         78|1080649.37|
|         34| 401419.75|
|         81|1053472.79|
|         28| 402124.62|
|         76|1058802.78|
|         26| 401506.97|
|         27| 399537.66|
|         44| 400598.05|
|         12| 401501.32|
|         91|1137372.73|
|         22| 402022.68|
|         93|1026006.06|
|         47|  398447.5|
|          1| 401044.25|
|         52| 733780.26|
|         13| 398917.98|
+-----------+----------+
only showing top 20 rows

--- 0.8072476387023926 seconds ---


In [21]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()
spark.table("home_sales").count()


33287

In [22]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

is_cached = spark.catalog.isCached('home_sales')

# Print the result
print(f"Is 'home_sales' table cached? {is_cached}")

Is 'home_sales' table cached? True


In [24]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
query = """
SELECT
    view AS view_rating,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    price >= 350000
GROUP BY
    view
"""
result = spark.sql(query)
result.show()

# Print runtime with caching
cached_runtime = time.time() - start_time
print(f"Runtime with caching: {cached_runtime} seconds")

# Clear the cache for comparison
spark.table("home_sales").unpersist()

# Without Cached Data
start_time = time.time()

# Running the same query
result_uncached = spark.sql(query)
result_uncached.show()

# Print runtime without caching
uncached_runtime = time.time() - start_time
print(f"Runtime without caching: {uncached_runtime} seconds")

# Showing the difference
difference = uncached_runtime - cached_runtime
print(f"Difference in runtime (uncached - cached): {difference} seconds")

+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|         31| 399856.95|
|         85|1056336.74|
|         65| 736679.93|
|         53|  755214.8|
|         78|1080649.37|
|         34| 401419.75|
|         81|1053472.79|
|         28| 402124.62|
|         76|1058802.78|
|         26| 401506.97|
|         27| 399537.66|
|         44| 400598.05|
|         12| 401501.32|
|         91|1137372.73|
|         22| 402022.68|
|         93|1026006.06|
|         47|  398447.5|
|          1| 401044.25|
|         52| 733780.26|
|         13| 398917.98|
+-----------+----------+
only showing top 20 rows

Runtime with caching: 0.7761011123657227 seconds
+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|         31| 399856.95|
|         85|1056336.74|
|         65| 736679.93|
|         53|  755214.8|
|         78|1080649.37|
|         34| 401419.75|
|         81|1053472.79|
|         28| 402124.62|
|         76|1058802.78|
|         26| 401506.97|


In [41]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

output_path = "/path/to/output/directory"

df.write.mode('overwrite').partitionBy("date_built").parquet(output_path)



In [47]:
# 11. Read the parquet formatted data.
# Define the path
parquet_path = "/path/to/output/directory"

# Read the partitioned Parquet data back into a DataFrame
df_from_parquet = spark.read.parquet(parquet_path)

# Display the first few rows of the DataFrame
df_from_parquet.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [48]:
# 12. Create a temporary table for the parquet data.
df_from_parquet.createOrReplaceTempView("temp_home_sales_parquet")

In [50]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
query_parquet = """
SELECT
    view AS view_rating,
    ROUND(AVG(price), 2) AS avg_price
FROM
    temp_home_sales_parquet
WHERE
    price >= 350000
GROUP BY
    view
"""

result_parquet = spark.sql(query_parquet)
result_parquet.show()

parquet_runtime = time.time() - start_time
print(f"Runtime with Parquet data: {parquet_runtime} seconds")



# cache the original DataFrame
spark.table("home_sales").cache()

# Measure runtime
start_time = time.time()

# Query on the cached DataFrame
query_cached = """
SELECT
    view AS view_rating,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    price >= 350000
GROUP BY
    view
"""

result_cached = spark.sql(query_cached)
result_cached.show()

cached_runtime = time.time() - start_time
print(f"Runtime with caching: {cached_runtime} seconds")

# Showing the difference
difference = parquet_runtime - cached_runtime
print(f"Difference in runtime (Parquet - cached): {difference} seconds")

+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|         31| 399856.95|
|         85|1056336.74|
|         65| 736679.93|
|         53|  755214.8|
|         78|1080649.37|
|         34| 401419.75|
|         81|1053472.79|
|         28| 402124.62|
|         76|1058802.78|
|         26| 401506.97|
|         27| 399537.66|
|         44| 400598.05|
|         12| 401501.32|
|         91|1137372.73|
|         22| 402022.68|
|         93|1026006.06|
|         47|  398447.5|
|          1| 401044.25|
|         52| 733780.26|
|         13| 398917.98|
+-----------+----------+
only showing top 20 rows

Runtime with Parquet data: 0.8750419616699219 seconds
+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|         31| 399856.95|
|         85|1056336.74|
|         65| 736679.93|
|         53|  755214.8|
|         78|1080649.37|
|         34| 401419.75|
|         81|1053472.79|
|         28| 402124.62|
|         76|1058802.78|
|         26| 401506

In [51]:
# 14. Uncache the home_sales temporary table.
spark.table("home_sales").unpersist()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [52]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales')

# Print the result
print(f"Is 'home_sales' table cached? {is_cached}")



Is 'home_sales' table cached? False
